## Zaawansowane Metody Inteligencji Obliczeniowej
# Zadanie domowe 2
### Prowadzący: Michał Kempka, Marek Wydmuch
### Autor: Daniel Zdancewicz 145317

## Wprowadzenie

Całe zadanie jest oparte o różne wersje środowiska `FrozenLake` ze znanej biblioteki OpenAI Gym (https://gym.openai.com), która agreguje różnego rodzaju środowiska pod postacią jednego zunifikowanego API.

Zapoznaj się z opisem środowiska (https://gym.openai.com/envs/FrozenLake-v0), a następnie zapoznaj się z kodem poniżej. Pokazuje on podstawy użytkowania API biblioteki Gym.

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod) o ile nie napisano gdzieś inaczej.

In [ ]:
# Zainstaluj bibliotekę OpenAI Gym w wersji 0.18.0
!pip install gym == 0.18.0

In [80]:
import gym

## Zad. 1 - Policy iteration + value iteration (10 pkt.)

W komórkach poniżej zaimplementuj algorytmy **iteracji polityki** oraz **iteracji wartości**, wyznaczające deterministyczną politykę dla środowiska FrozenLake.

Odpowiedź na pytania wykonując odpowiednie eksperymenty (zostaw output odpowiednich komórek na poparcie swoich twierdzeń):
- Jak zmiana współczynniku `gamma` wpływa na wynikową politykę?
- Jak stochastyczność wpływa na liczbę iteracji potrzebnych do zbiegnięcia obu algorytmów oraz wynikową politykę?

#### Uwaga: nie zmieniaj nazwy funkcji `policy_iteration` i `value_iteration`, ani ich argumentów. Nie dopisuj do komórek z funkcjami innego kodu. Może zdefiniować funkcje pomocnicze dla danej funkcji w tej samej komórce (sprawdzarka wyciągnie ze zgłoszonego notebooka wyłącznie komórki zawierającą funkcje `policy_iteration` i `value_iteration` do sprawdzenia, kod w innych komórkach nie będzie widziany przez sprawdzarkę!)

Odpowiedzi:
1. Współczynnik `gamma` ustawiony w przedziale 0,0-0,1 lub o wartości 1.0 powoduje błędne polityki o złych liczbach iteracji, które powodują przegraną ciągłą przegraną.
2. Stochastyczność wpływa na:
- Liczba iteracji potrzebnych do zbiegnięcia:
  - Stochastyczne — Potrzebujemy więcej iteracji przy większym gamma do zbiegnięcia. Może to wynikać ze względu na otrzebę uwzględnienia prawdopodobieństwa przejść pomiędzy stanami, co każe agentowi myśleć o ryzku związanymi z akcjami.
  - Deterministyczne — zbiegają szybciej ze względu na brak potrzeby przewidywania przez agenta ryzyka swoich akcji.
- Wynikowa polityka:
  - Stochastyczne — Zdaje się ostrożna, agent możliwie uwzględnia ryzyko związane ze ślizganiem się na lodzie. Przez co możliwie, żę będzie rozważać ścieżki, które prowadzą do celu z mniejszym ryzykiem, ale są dłuższe.
  - Deterministyczne — Agent może zawsze wybrać najkrótszą ścieżkę do celu ze względu na pewność wykonanych akcji przez agenta.

In [81]:
def evaluate_empirically(env, pi, episodes=1000, max_actions=100):
  mean_reward = 0
  for episode in range(episodes):
    state = env.reset()
    total_reward = 0

    for _ in range(max_actions):
      state, reward, termination, _ = env.step(pi[state])
      total_reward += reward
      if termination: break
    mean_reward = mean_reward + 1 / (episode + 1) * (total_reward - mean_reward)
  return mean_reward

In [82]:
def policy_iteration(P, gamma, delta=0.001):
  """
  Argumenty:
      P — model przejścia, gdzie P[s][a] == [(probability, nextstate, reward, done), ...]
      gamma — współczynnik dyskontujący
      delta — tolerancja warunku stopu
  Zwracane wartości:
      V — lista o długości len(P) zawierający oszacowane wartość stanu s: V[s]
      pi — lista o długości len(P) zawierający wyznaczoną deterministyczną politykę - akcję dla stanu s: pi[s]
      i — ilość iteracji algorytmu po wszystkich stanach
  """
  def evaluate():
    while True:
      score = 0
      for state in states:
        value = V[state]
        action = policy[state]
        V[state] = sum(
          probability * (reward + gamma * V[next]) for probability, next, reward, _ in P[state][action]
        )
        score = max(score, abs(value - V[state]))
      if score < delta: break

  def improve():
    is_stable = True
    for state in states:
      previous = policy[state]
      actions = range(len(P[state]))
      policy[state] = max(actions, key=lambda action: sum(
        probability * (reward + gamma * V[next]) for probability, next, reward, _ in P[state][action]
      ))

      if previous != policy[state]: is_stable = False
    return is_stable

  V = [0] * len(P)
  policy = [0] * len(P)
  states = range(len(P))
  iterations = 0

  while True:
    iterations += 1
    evaluate()
    is_stable = improve()
    if is_stable: break

  return V, policy, iterations


In [83]:
env = gym.make('FrozenLake-v0', is_slippery=False)
P = env.P
gamma = 0.5
V, policy, iterations = policy_iteration(P, gamma)

print(f"Wartości stanów (V):\n{V}")
print(f"Polityka (pi):\n{policy}")
print(f"Ilość iteracji: {iterations}")

evaluate_empirically(env, policy)


Wartości stanów (V):
[0.03125, 0.0625, 0.125, 0.0625, 0.0625, 0.0, 0.25, 0.0, 0.125, 0.25, 0.5, 0.0, 0.0, 0.5, 1.0, 0.0]
Polityka (pi):
[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]
Ilość iteracji: 7


1.0

In [84]:
def value_iteration(P, gamma, delta=0.001):
  """
  Argumenty:
      P — model przejścia, gdzie P[s][a] == [(probability, nextstate, reward, done), ...]
      gamma — współczynnik dyskontujący
      delta — tolerancja warunku stopu
  Zwracane wartości:
      Q — lista o długości len(P) zawierający listy z oszacowanymi wartościami dla stanu s i akcji a: Q[s][a]
      pi — lista o długości len(P) zawierający wyznaczoną deterministyczną politykę - akcję dla stanu s: pi[s]
      i — ilość iteracji algorytmu po wszystkich stanach
  """
  def evaluate():
    score = 0
    for state in states:
      for action in range(len(P[state])):
        q = Q[state][action]
        Q[state][action] = sum([p * (r + gamma * max(Q[s_])) for p, s_, r, _ in P[state][action]])
        score = max(score, abs(q - Q[state][action]))
    return score

  def extract():
    return [Q[state].index(max(Q[state])) for state in states]

  states = P.keys()
  Q = [[0] * len(P[state]) for state in states]
  iterations = 0

  while True:
    iterations += 1
    score = evaluate()
    if score < delta: break

  pi = extract()
  return Q, pi, iterations

In [85]:
env = gym.make('FrozenLake-v0', is_slippery=False)
P = env.P
gamma = 0.5
Q, policy, iterations = value_iteration(P, gamma)

print(f"Wartości stanów i akcji (Q):")
print('[', *Q, ']', sep=',\n')
print(f"Polityka (pi):\n{policy}")
print(f"Ilość iteracji: {iterations}")
evaluate_empirically(env, policy)

Wartości stanów i akcji (Q):
[,
[0.015625, 0.03125, 0.03125, 0.015625],
[0.015625, 0.0, 0.0625, 0.03125],
[0.03125, 0.125, 0.03125, 0.0625],
[0.0625, 0.0, 0.03125, 0.03125],
[0.03125, 0.0625, 0.0, 0.015625],
[0.0, 0.0, 0.0, 0.0],
[0.0, 0.25, 0.0, 0.0625],
[0.0, 0.0, 0.0, 0.0],
[0.0625, 0.0, 0.125, 0.03125],
[0.0625, 0.25, 0.25, 0.0],
[0.125, 0.5, 0.0, 0.125],
[0.0, 0.0, 0.0, 0.0],
[0.0, 0.0, 0.0, 0.0],
[0.0, 0.25, 0.5, 0.125],
[0.25, 0.5, 1.0, 0.25],
[0.0, 0.0, 0.0, 0.0],
]
Polityka (pi):
[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]
Ilość iteracji: 8


1.0

In [162]:
# Eksperyment wpływu `gamma`.
env = gym.make('FrozenLake-v0', is_slippery=False)
P = env.P
gammas = [0.0, 0.1, 0.5, 0.9, 0.99, 1]

print("Policy Iteration:")
for gamma in gammas:
  V, pi, i = policy_iteration(P, gamma)
  print(f"Gamma: {gamma}, Polityka: {pi}, Liczba iteracji: {i}")
  print(f"Wynik: {evaluate_empirically(env, pi)}")

print("\nValue Iteration:")
for gamma in gammas:
  Q, pi, i = value_iteration(P, gamma)
  print(f"Gamma: {gamma}, Polityka: {pi}, Liczba iteracji: {i}")
  print(f"Wynik: {evaluate_empirically(env, pi)}")


Policy Iteration:
Gamma: 0.0, Polityka: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0], Liczba iteracji: 2
Wynik: 0.0
Gamma: 0.1, Polityka: [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0], Liczba iteracji: 7
Wynik: 1.0
Gamma: 0.5, Polityka: [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0], Liczba iteracji: 7
Wynik: 1.0
Gamma: 0.9, Polityka: [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0], Liczba iteracji: 7
Wynik: 1.0
Gamma: 0.99, Polityka: [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0], Liczba iteracji: 7
Wynik: 1.0
Gamma: 1, Polityka: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], Liczba iteracji: 8
Wynik: 0.0

Value Iteration:
Gamma: 0.0, Polityka: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0], Liczba iteracji: 2
Wynik: 0.0
Gamma: 0.1, Polityka: [0, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0], Liczba iteracji: 5
Wynik: 0.0
Gamma: 0.5, Polityka: [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0], Liczba iteracji: 8
Wynik: 1.0
Gamma: 0.9, Polityka: [1, 2, 1, 0, 1, 0, 1, 0, 

In [171]:
# Eksperyment środowiska deterministycznego kontra stochastycznego.
env_deterministic = gym.make('FrozenLake-v0', is_slippery=False)
env_stochastic = gym.make('FrozenLake-v0', is_slippery=True)
P_deterministic = env_deterministic.P
P_stochastic = env_stochastic.P

gammas = [0.0, 0.1, 0.5, 0.9, 0.99, 1]

print("Policy Iteration:")
for gamma in gammas:
  V_det, pi_det, i_det = policy_iteration(P_deterministic, gamma)
  V_sto, pi_sto, i_sto = policy_iteration(P_stochastic, gamma)
  print(f"Deterministyczny: Polityka: {pi_det}, Liczba iteracji: {i_det}")
  print(f"Wynik: {evaluate_empirically(env_deterministic, pi_det)}")
  print(f"Stochastyczny: Polityka: {pi_sto}, Liczba iteracji: {i_sto}")
  print(f"Wynik: {evaluate_empirically(env_stochastic, pi_sto)}")

print("\nValue Iteration:")
for gamma in gammas:
  Q_det, pi_det, i_det = value_iteration(P_deterministic, gamma)
  Q_sto, pi_sto, i_sto = value_iteration(P_stochastic, gamma)
  print(f"Deterministyczny: Polityka: {pi_det}, Liczba iteracji: {i_det}")
  print(f"Wynik: {evaluate_empirically(env_deterministic, pi_det)}")
  print(f"Stochastyczny: Polityka: {pi_sto}, Liczba iteracji: {i_sto}")
  print(f"Wynik: {evaluate_empirically(env_stochastic, pi_sto)}")


Policy Iteration:
Deterministyczny: Polityka: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0], Liczba iteracji: 2
Wynik: 0.0
Stochastyczny: Polityka: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], Liczba iteracji: 2
Wynik: 0.0
Deterministyczny: Polityka: [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0], Liczba iteracji: 7
Wynik: 1.0
Stochastyczny: Polityka: [1, 3, 2, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], Liczba iteracji: 6
Wynik: 0.474
Deterministyczny: Polityka: [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0], Liczba iteracji: 7
Wynik: 1.0
Stochastyczny: Polityka: [1, 3, 2, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], Liczba iteracji: 5
Wynik: 0.41300000000000003
Deterministyczny: Polityka: [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0], Liczba iteracji: 7
Wynik: 1.0
Stochastyczny: Polityka: [0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], Liczba iteracji: 6
Wynik: 0.727000000000001
Deterministyczny: Polityka: [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0], Liczba iteracji: 7
W

## Zad. 2 - Monte Carlo (10 pkt.)
W komórce poniżej zaimplementuj metodę **On-policy Monte Carlo** dla polityki epsilon-greedy.
Zakładamy, że model przejść nie jest w tym wypadku dla nas dostępny,
dlatego możesz używać wyłącznie metod `env.reset()` i `env.step()`
w swojej implementacji, w celu wygenerowania nowego epizodu.

- Zaproponuj warunek stopu dla swojej implementacji.
- Jaki jest wpływ epsilony na działanie algorytmu?
- Jaka prosta modyfikacja nagród środowiska przyśpieszyłaby odkrywanie dobrej polityki? Zmodyfikuj env.P i zademonstruj.

Tip: z racji, że env.P jest dostępne, możesz porównać wyniki `on_policy_eps_greedy_monte_carlo` ze wynikami `value_iteration`. 

#### Uwaga: nie zmieniaj nazwy funkcji `on_policy_eps_greedy_monte_carlo`, ani jej pierwszych argumentów (możesz dodać nowe argumenty z wartościami domyślnymi). Nie dopisuj do komórki z funkcją innego kodu. Może zdefiniować funkcje pomocnicze dla funkcji w tej samej komórce (sprawdzarka wyciągnie ze zgłoszonego notebooka wyłącznie komórkę zawierającą funkcję `on_policy_eps_greedy_monte_carlo` do sprawdzenia, kod w innych komórkach nie będzie widziany przez sprawdzarkę!).

Odpowiedź: Miejsce na Twoje odpowiedzi

In [224]:
import random
from collections import defaultdict

def modify_rewards(P, goal, reward_step=-0.01, reward_goal=1.0):
  P = P.copy()
  for state in range(len(P)):
    for action in range(len(P[state])):
      P[state][action] = [
        (probability, next, next == goal and reward_goal or reward_step, done)
        for (probability, next, reward, done) in P[state][action]
      ]
  return P

def on_policy_eps_greedy_monte_carlo(env, eps, gamma, max_episodes=10000):
  """
  Argumenty:
      env — środowisko implementujące metody `reset()` oraz `step(action)`
      eps — współczynnik eksploracji
      gamma — współczynnik dyskontujący
  Zwracane wartości:
      Q — lista o długości len(P) zawierający listy z oszacowanymi wartościami dla stanu s i akcji a: Q[s][a]
      pi — lista o długości len(P) zawierający wyznaczoną deterministyczną (zachłanną) politykę - akcję dla stanu s: pi[s]
      i — ilość epizodów wygenerowanych przez algorytm
  """
  def generate_episodes():
    nonlocal iterations
    while (iterations := iterations + 1) < max_episodes:
      state = env.reset()
      episode = []
      termination = False

      while not termination:
        action = pi[state] if random.random() > eps else random.randrange(len(P[state]))
        next_state, reward, termination, _ = env.step(action)
        episode.append((state, action, reward))
        state = next_state

      yield episode

  def evaluate(episode):
    result = 0

    for (state, action, reward) in reversed(episode):
      result = gamma * result + reward

      counts[state][action] += 1
      Q[state][action] += (result - Q[state][action]) / counts[state][action]

  def extract():
    return [Q[state].index(max(Q[state])) for state in range(len(Q))]

  P = env.P
  states = range(len(P))
  pi = [0] * len(states)
  Q = [[0] * len(P[s]) for s in states]
  counts = [[0] * len(P[state]) for state in states]
  iterations = 0

  episode_it = generate_episodes()
  while episode := next(episode_it, None):
    evaluate(episode)
    pi = extract()

  return Q, pi, iterations

def on_policy_eps_greedy_monte_carlo(env, eps, gamma, max_episodes=5000, improvement_threshold=0.001):
  def eps_greedy_policy(Q, state, eps):
    if random.random() < eps:
      return random.randint(0, len(Q[state]) - 1)
    return max(range(len(Q[state])), key=lambda a: Q[state][a])

  pi = [0] * len(env.P)
  Q = defaultdict(lambda: [0] * len(env.P[0]))
  returns = defaultdict(list)
  i = 0

  while i < max_episodes:
    i += 1
    episode = []
    state = env.reset()
    done = False

    while not done:
      action = eps_greedy_policy(Q, state, eps)
      next_state, reward, done, _ = env.step(action)
      episode.append((state, action, reward))
      state = next_state

    G = 0
    visited = set()
    for t in reversed(range(len(episode))):
      state, action, reward = episode[t]
      G = gamma * G + reward

      if (state, action) not in visited:
        visited.add((state, action))
        returns[(state, action)].append(G)
        Q[state][action] = sum(returns[(state, action)]) / len(returns[(state, action)])
        old_action = pi[state]
        pi[state] = max(range(len(Q[state])), key=lambda a: Q[state][a])

        if abs(Q[state][old_action] - Q[state][pi[state]]) > improvement_threshold:
          break
  return Q, pi, i

In [232]:
env = gym.make('FrozenLake-v0', is_slippery=True)
epsilon = 0.7
gamma = 0.9
max_episodes = 10_000
env.P = modify_rewards(env.P, goal=15, reward_goal=1, reward_step=-0.02)
Q, pi, i = on_policy_eps_greedy_monte_carlo(env, epsilon, gamma, max_episodes)
print(f"Wartości stanów i akcji (P):")
print(*env.P.items(), sep='\n')
print(f"Wartości stanów i akcji (Q):\n{Q}")
print(f"Polityka (pi):\n{pi}")
print(f"Ilość epizodów: {i}")
evaluate_empirically(env, pi)

Wartości stanów i akcji (P):
(0, {0: [(0.3333333333333333, 0, -0.02, False), (0.3333333333333333, 0, -0.02, False), (0.3333333333333333, 4, -0.02, False)], 1: [(0.3333333333333333, 0, -0.02, False), (0.3333333333333333, 4, -0.02, False), (0.3333333333333333, 1, -0.02, False)], 2: [(0.3333333333333333, 4, -0.02, False), (0.3333333333333333, 1, -0.02, False), (0.3333333333333333, 0, -0.02, False)], 3: [(0.3333333333333333, 1, -0.02, False), (0.3333333333333333, 0, -0.02, False), (0.3333333333333333, 0, -0.02, False)]})
(1, {0: [(0.3333333333333333, 1, -0.02, False), (0.3333333333333333, 0, -0.02, False), (0.3333333333333333, 5, -0.02, True)], 1: [(0.3333333333333333, 0, -0.02, False), (0.3333333333333333, 5, -0.02, True), (0.3333333333333333, 2, -0.02, False)], 2: [(0.3333333333333333, 5, -0.02, True), (0.3333333333333333, 2, -0.02, False), (0.3333333333333333, 1, -0.02, False)], 3: [(0.3333333333333333, 2, -0.02, False), (0.3333333333333333, 1, -0.02, False), (0.3333333333333333, 0, -0.

-0.06596